# Frist chat using Azure OpenAI SDK
This sample notebook will let you have a first chat with GPT-4o.

## Prerequisites
1. You have [set up the environment](../README.md).
1. You have an Azure OpenAI deployment.


## Create environment variables for Azure OpenAI endpoint and key
- **Endpoint**: The endpoint is the URL that you use to access the Azure OpenAI service. It typically follows this format: `https://<your-resource-name>.openai.azure.com`
 
- **Key**: The key is a secret token used to authenticate your API requests. 

In the Azure AI project **Overview**, select **Azure OpenAI**, you can find the endpoint, and key.
![endpoint-key](../media/first_chat/endpoint-key.png)

In [ ]:
%env AZURE_OPENAI_ENDPOINT=REPLACE_WITH_YOUR_ENDPOINT

In [ ]:
%env AZURE_OPENAI_API_KEY=REPLACE_WIHT_YOUR_KEY

## First chat with LLMs

In AI project **Models*Endpoints**, you can find the **api version**.
![api-version](../media/first_chat/api-version.png)

In [ ]:
import os
from openai import AzureOpenAI

deployment_name = "gpt-4o"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Introduce Azure OpenAI within 200 words"}
    ]
)

print(response.choices[0].message.content)

## Chat including a local image input

In [ ]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Example usage
image_path = '../media/first_chat/cat.jpg'
data_url = local_image_to_data_url(image_path)
print("Data URL:", data_url)

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture in a vivid way within 100 words:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }
        ] } 
    ],
    max_tokens=2000 
)

print(response.choices[0].message.content)